# Full state tomography

- [Imports](#imports)
- [Pauli Matrices](#pauli-matrices)
- [From circuit to tomography](#from-circuit-to-tomography)
- [Backend choice and Execution](#backend-choice-and-tomography-execution)
- [Density Matrix recreation](#density-matrix-recreation)
- [Fidelity calculation](#fidelity-caluclation)

## Imports

[Back to top](#full-state-tomography)

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

from qiskit import QuantumCircuit, assemble
from qiskit.circuit import Gate, Qubit
from qiskit.visualization import plot_state_city

from umz_qiskit_backend.qiskit_backend import UmzBackend, RedTrapBackend, UmzSimulatorBackend

from IPython.display import display, Latex
from collections.abc import Iterable

def is_iterable(obj):
    return isinstance(obj, Iterable)

def format_imag_part(imag_part, digits) -> str:
    if imag_part == 1:
        return "i"
    elif imag_part == -1:
        return "-i"
    elif imag_part == 0:
        return "0"
    return f"{np.round(imag_part, digits)}i"

def format_real_part(real_part, digits) -> str:
    if real_part == 1:
        return "1"
    elif real_part == -1:
        return "-1"
    elif real_part == 0:
        return "0"
    return f"{np.round(real_part, digits)}"

def complex_format(value, digits = 2) -> str:
    if isinstance(value, complex):
        real_part = np.round(value.real, digits) if value.real != 0 else 0
        imag_part = np.round(value.imag, digits) if value.imag != 0 else 0
        if real_part == 0:
            return format_imag_part(imag_part, digits)
        else:
            real_part_repr = format_real_part(real_part, digits)
            imag_part_repr = format_imag_part(imag_part, digits)
            if imag_part_repr == "0":
                return real_part_repr
            elif imag_part_repr[0] == "-":
                return f"{real_part_repr} - {imag_part_repr[1:]}"
            return f"{real_part_repr} + {imag_part_repr}"
    elif isinstance(value, (int, float)):
        return "0" if value == 0 else f"{np.round(value, digits)}"
    else:
        return str(value)

def print_matrix(matrix: np.ndarray):
    matrix_latex = "\\begin{pmatrix}\n"
    for row in matrix:
        if not is_iterable(row):
            row_str = complex_format(row)
        else:
            row_str = " & ".join([complex_format(elem) for elem in row])
        matrix_latex += f"{row_str} \\\\\n"
    matrix_latex += "\\end{pmatrix}"

    display(Latex(matrix_latex))

## Pauli matrices

[Back to top](#full-state-tomography)

Define the 4 Pauli matrices:
$$
\sigma_0 = \begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix},
\quad
\sigma_x = \begin{pmatrix}
0 & 1 \\
1 & 0
\end{pmatrix},
\quad
\sigma_y = \begin{pmatrix}
0 & -i \\
i & 0
\end{pmatrix},
\quad
\sigma_z = \begin{pmatrix}
1 & 0 \\
0 & -1
\end{pmatrix}
$$

In [2]:
# Define the pauli matrices
s_0 = np.identity(2, dtype=complex)
s_x = np.array([
    [0+0j, 1+0j],
    [1+0j, 0+0j]
])
s_y = np.array([
    [0+0j, 0-1j],
    [0+1j, 0+0j]
])
s_z = np.array([
    [1+0j, 0+0j],
    [0+0j, -1+0j]
])

## From circuit to tomography

[Back to top](#full-state-tomography)

In [3]:
def basis_change(circ: QuantumCircuit, bases: list[str]) -> QuantumCircuit:
    """
    Apply basis change rotations to a QuantumCircuit.

    This function takes a QuantumCircuit and a list of basis strings
    ('x', 'X', 'y', 'Y', 'z', 'Z') and applies rotations accordingly
    to each qubit in the circuit.

    Parameters
    ----------
    circ : QuantumCircuit
        The input circuit to which basis rotations will be applied.
    bases : List[str]
        A list of basis strings ('x', 'X', 'y', 'Y', 'z', 'Z') representing
        the rotations to be applied to each qubit.

    Returns
    -------
    QuantumCircuit
        The input circuit with the specified basis change rotations.

    Raises
    ------
    AssertionError
        If the number of qubits in the circuit does not match the length of 'bases'.

    AssertionError
        If the circuit contains classical bits (num_clbits > 0).

    AssertionError
        If an invalid basis name is encountered.

    """
    assert circ.num_qubits == len(bases)
    assert circ.num_clbits == 0
    rotated_basis_circ = QuantumCircuit(circ.num_qubits)
    rotated_basis_circ = rotated_basis_circ.compose(circ)
    for q, basis in zip(rotated_basis_circ.qubits, bases):
        if basis == 'x' or basis == 'X':
            rotated_basis_circ.r(np.pi / 2, -np.pi / 2, q)
        elif basis == 'y' or basis == 'Y':
            rotated_basis_circ.rx(np.pi / 2, q)
        elif basis == 'z' or basis == 'Z':
            pass
        else:
            assert False, "Invalid basis name '%s'" % (basis)
    rotated_basis_circ.measure_all()
    return rotated_basis_circ

def do_tomography(circ: QuantumCircuit, basis: list, backend: UmzBackend, shots: int):
    """
    Perform state tomography on a QuantumCircuit in a specified basis.

    This function takes a QuantumCircuit, a basis (a list of basis strings),
    a backend, and the number of shots, and returns a dictionary with
    measurement results for the given basis.

    Parameters
    ----------
    circ : QuantumCircuit
        The input circuit for which state tomography will be performed.
    basis : List[str]
        A list of basis strings ('x', 'X', 'y', 'Y', 'z', 'Z') representing
        the basis in which the tomography will be performed.
    backend : UmzBackend
        The backend used for running the circuit and obtaining results.
    shots : int
        The number of shots to run on the backend.

    Returns
    -------
    Dict[str, int]
        A dictionary containing the measurement outcomes ('00', '01', etc.)
        and their corresponding counts.

    """
    print("Measuring basis", basis)

    # Perform the basis rotation on the circuit
    rotated_basis_circ = basis_change(circ, basis)

    # Run the circuit and get the results
    job = backend.run([rotated_basis_circ], shots=shots)
    
    # binary data as dict: {'00': counts of 00, '01': counts of 01, ...}
    data = job.result()['result'][0]['measurements']

    return data

def do_full_state_tomography(circ: QuantumCircuit, backend: UmzBackend, shots: int = 1000) -> dict:
    """
    Perform full state tomography on a QuantumCircuit.

    This function performs full state tomography on a QuantumCircuit in
    the X, Y, and Z bases for each qubit. It returns a dictionary containing
    the measurement outcomes for each basis combination.

    Parameters
    ----------
    circ : QuantumCircuit
        The input circuit for which full state tomography will be performed.
    backend : UmzBackend
        The backend used for running the circuit and obtaining results.
    shots : int, optional
        The number of shots to run on the backend, by default 1000.

    Returns
    -------
    Dict[str, Dict[str, int]]
        A nested dictionary where the outer keys are basis combinations
        (e.g., 'XZ'), and the inner dictionaries contain measurement
        outcomes ('00', '01', etc.) and their corresponding counts.

    """
    print("Tomography on:")
    print(circ)
    # Define the three bases for each qubit
    bases = ['X', 'Y', 'Z']

    # Calculate all bases combinations (3^n)
    num_qubits = circ.num_qubits
    bases = list(product(*[bases for _ in range(num_qubits)]))
    print("Bases for full tomography:", bases)
    
    data_dict = {}
    '''Dictionary to hold the measurement data for each basis combination'''
    
    # Perform the state tomography for each base combination
    for i, basis in enumerate(bases):
        print(f'{i + 1}/{len(bases)}', end=" ")
        data = do_tomography(circ, basis, backend, shots)
        basis_key = ''
        for base in basis:
            basis_key += base
        data_dict[basis_key] = data
    
    return data_dict

## Backend choice and tomography execution

[Back to top](#full-state-tomography)

In [6]:
circ = QuantumCircuit(2)
circ.rx(np.pi, 0)

with open('C:\\access', 'r') as f:
    user = f.readline().strip()
    pw = f.readline().strip()

shots = 1000
backend = UmzSimulatorBackend(email=user, password=pw)
tomography_data = do_full_state_tomography(circ, backend, shots)

# convert counts to probabilities
results = {base_pair: {state: counts / shots for state, counts in measurement_result.items()} for base_pair, measurement_result in tomography_data.items()}
with open('3QB_test_sim02.json', 'w') as f:
    results = json.load(f)
print(results)

## Density matrix recreation

[Back to top](#full-state-tomography)

Density matrix recreation can be done from a complete state tomography where each of the $3^n$ ($n$ as number of qubits) basis are measured. From there we can extract the expectation values for the Pauli operators:  
Measuring in the $\underbrace{ZZ...}_{n \text{ times}}$ basis gives results for all $2^n$ possible $n - $ qubit states. From these binary probabilities we need to get to the expectation values which can be done by looking at the connection from probability to Pauli matrix expectation value which can be visualized with the Bloch sphere:

<div style="text-align: center;">
    <img src="./bloch.png" style="display: inline-block;">
</div>

Measuring in $Z$ with a probability $0.5$ for $\ket{\uparrow}$ and $0.5$ for $\ket{\downarrow}$ results in an expectation value of
$$P_{\ket{\uparrow}} - P_{\ket{\downarrow}} = 0.5 - 0.5 = 0$$
for $\sigma_z$. For each Pauli matrix, one can visualize the Bloch sphere to see how the expectation value is connected to the respective measurement result. For simplicity, I will refer to $\ket{1}$ as a "bright" result for each base. This means that in $Z$, $\ket{1}$ corresponds to $\ket{\uparrow}$ while in $X$,  $\ket{1}$ corresponds to $\ket{+}$ which can be written in the logical basis ($Z$) as $1/\sqrt{2} (\ket 0 + \ket 1)$. Things easily get very cluttered here so be cautious. Because the tomography measurement data we obtain gives only bright and dark results ($\ket{1}$ and $\ket{0}$) for each basis measurement we can express the respective expectation values using this formalism.

$$ \langle \sigma_0 \rangle = P_{\ket{1}} + P_{\ket{0}}, \quad \langle \sigma_i \rangle = P_{\ket{1}} - P_{\ket{0}} \text{  for base }i \in \{1, 2 ,3\}$$

From there we can calculate all expectation values for composite states $\langle \sigma_{i_1} \otimes \sigma_{i_2} \otimes ... \otimes \sigma_{i_n} \rangle$ by composing the expectation values for the individual qubits.
$$
\langle \sigma_{i_1} \otimes \sigma_{i_2} \otimes ... \otimes \sigma_{i_n} \rangle \\
= \prod_{j = 1}^n \langle \sigma_{i_j} \rangle = \prod_{j = 1}^n P_{\ket{1}} + (-1)^{s(i_j)} P_{\ket{0}}
$$

where I define

$$
s(i) = \begin{cases}
0 & \text{if } i = 0, \\
1 & \text{otherwise}.
\end{cases}
$$

which catches the sign difference for $\sigma_0$ and other $\sigma_i$ for $i \neq 0$. In the following code, these expectation values will be calculated by the function `alpha`.

Reference: [J. B. Altepeter, D. F. V. James, and P. G. Kwiat, Quantum State Tomography](http://research.physics.illinois.edu/QI/Photonics/tomography-files/tomo_chapter_2004.pdf)

In [7]:
def alpha(*indices):
    """Calculation of <sigma_i x sigma_j> (Expectation value)

    Parameters
    ----------
    *indices : list[int]
        Pauli matrix indices for the qubits.

    Returns
    -------
    float
        Expectation value of a sigma_i x sigma_j measurement.
    """
    m = {
        0: 'Z',
        1: 'X',
        2: 'Y',
        3: 'Z',
    }
    # get the result for that basis measurement
    basis = ''
    for i in indices:
        basis += m[i]
    res = results[basis]

    # from the probabilities obtain the coefficient = expectation value of
    # the pauli matrix combination
    coeff = 0
    for bin_outcome in res.keys():
        sign = 1
        for i, bit in enumerate(bin_outcome):
            if bit == '0' and indices[i] != 0:
                sign *= -1
        
        coeff += sign * res.get(bin_outcome, 0)
    return coeff

def kronecker_recursive(matrices: list[np.ndarray]):
    """Recursively calculates the Kronecker product of a list of matrices.

    Parameters
    ----------
    matrices : list of numpy.ndarray
        List of matrices to be Kronecker multiplied.

    Returns
    -------
    numpy.ndarray
        Resulting Kronecker product of the input matrices.
    """
    if len(matrices) == 1:
        return matrices[0]
    return np.kron(matrices[0], kronecker_recursive(matrices[1:]))

def calculate_rho(n: int, paulis: list[np.ndarray]):
    """Calculates the density matrix rho for a given number of qubits.

    This function calculates the density matrix rho based on the specified value
    of qubits n and a list of Pauli matrices.

    Parameters
    ----------
    n : int
        Number of qubits n that determines the number of loops and Kronecker products.
    paulis : list of numpy.ndarray
        List of Pauli matrices used in the calculation.

    Returns
    -------
    numpy.ndarray
        The calculated density matrix rho.
    """
    rho = np.zeros((2 ** n, 2 ** n), dtype=complex)

    for indices in product(range(4), repeat=n):
        alpha_value = alpha(*indices)
        rho += alpha_value * kronecker_recursive([paulis[i] for i in indices])

    rho *= 1 / 2 ** n  # Normalization
    return np.flipud(np.fliplr(rho)) # Invert states: 010 -> 101

num_qb = circ.num_qubits
paulis = [s_0, s_x, s_y, s_z]

# Calculate the density matrix
rho = np.round(calculate_rho(num_qb, paulis), 2)

print('Density matrix:\n')
print_matrix(rho)

# Calculate the eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(rho)

# Find the index of the highest eigenvalue
max_index = np.argmax(eigenvalues)

# Get the corresponding eigenvector (state vector)
state_vector = eigenvectors[:, max_index]

# Normalize the state vector to make it a unit vector (if needed)
state_vector /= np.linalg.norm(state_vector)

print("State vector:")
print_matrix(state_vector)

plot_state_city(rho, color=['midnightblue', 'crimson'], title="Density matrix")


NameError: name 'results' is not defined

## Fidelity caluclation

[Back to Top](#full-state-tomography)

At first we calculate the expected state using the following gate definitions:

$$
R_X(\vartheta) = \exp\left(-i \frac{\vartheta}{2} X\right) =
\begin{pmatrix}
\cos(\vartheta / 2) & - i \sin(\vartheta / 2) \\
- i \sin(\vartheta / 2) & \cos(\vartheta / 2) \\
\end{pmatrix}
$$

and

$$
R_{ZZ}(\vartheta) = \exp\left(-i \frac{\vartheta}{2} Z{\otimes}Z\right) =
\begin{pmatrix}
    e^{-i \vartheta / 2} & 0 & 0 & 0 \\
    0 & e^{i \vartheta / 2} & 0 & 0 \\
    0 & 0 & e^{i \vartheta / 2} & 0 \\
    0 & 0 & 0 & e^{-i \vartheta / 2}
\end{pmatrix} = e^{i \vartheta / 2} \cdot
\begin{pmatrix}
    e^{-i \vartheta} & 0 & 0 & 0 \\
    0 & 1 & 0 & 0 \\
    0 & 0 & 1 & 0 \\
    0 & 0 & 0 & e^{-i \vartheta}
\end{pmatrix}
$$

according to qiskit: [$R_X$](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RXGate.html), [$R_{ZZ}$](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RZZGate.html)

$X$ and $Z$ refer to the Pauli matrices $\sigma_X$ and $\sigma_Z$.

This results in the "Bell state operator"
$$
\hat{B} = [R_X(\pi / 2) \otimes R_X(\pi / 2)] \cdot R_{ZZ}(\pi / 2) \cdot [R_X(\pi / 2) \otimes R_X(\pi / 2)] = 
\begin{pmatrix}
-1 & 0 & 0 & -i \\
0 & 1 & -i & 0 \\
0 & -i & 1 & 0 \\
-i & 0 & 0 & -1 \\
\end{pmatrix}
$$

Using the input state $\ket{00}$ with the matrix representation $s_0 = \begin{pmatrix}
1 \\
0 \\
0 \\
0 \\
\end{pmatrix}$ we obtain the theoretical output state
$$
\hat{B} \cdot s_0 = o_0 = -i / \sqrt 2
\begin{pmatrix}
1 \\
0 \\
0 \\
i \\
\end{pmatrix}
$$

The fidelity can now be calculated from the density matrix by calculating the matrix element when going from $o_0$ to $o_0$:

$$
f = o_0^\dagger \cdot \hat \rho \cdot o_0
$$

In [15]:
def get_binary_representation(integer: int, size: int) -> str:
    """Converts an integer to its binary representation with leading zeros, ensuring a fixed size.

    This function takes an integer and converts it to its binary representation.
    Leading zeros are added to the binary representation to ensure that the resulting
    string always has the specified size. If the binary representation is too long to fit
    within the desired size, a ValueError exception is raised.

    Parameters
    ----------
    integer : int
        The integer to be converted to binary.
    size : int
        The desired size of the binary representation string.

    Returns
    -------
    str
        The binary representation of the integer as a string with leading zeros.

    Raises
    ------
    ValueError
        If the binary representation is too large for the given size.

    Examples
    --------
    >>> get_binary_representation(5, 8)
    '00000101'

    >>> get_binary_representation(10, 6)
    '010101'

    >>> get_binary_representation(255, 8)
    '11111111'
    """
    binary_string = bin(integer)[2:]  # Remove '0b' prefix from binary string

    if len(binary_string) > size:
        raise ValueError("Binary representation too large for the given size")

    padded_binary_string = binary_string.rjust(size, '0')
    return padded_binary_string

def calculate_new_col(col: int, index_map: dict) -> int:
    """Calculates the new column for column col, given an index_map
    which maps old qubit indices to their new qubit indices.

    The column in a density matrix shaped matrix (2^n x 2^n) that transforms
    a representation of a state vector (size 2^n) where element i represents the
    qubit state i.to_binary(), e.g. i = 3 corresponds to state 0..011. The
    index_map maps each position in the binary representation to a new position.
    The resulting binary value is then converted back to a decimal value which
    corresponds to the new column that is returned.

    Parameters
    ----------
    col : int
        Old column in a matrix of size 2^n x 2^n.
    index_map : dict
        Map which maps the old qubit index to the new qubit index.

    Returns
    -------
    int
        New column.
    """
    num_qb = len(index_map.values())
    
    # get the binary representation of the old column
    state = get_binary_representation(col, num_qb)

    # sort the index_map to be ascending in the values to
    # simply iterate over it and build up the new state
    sorted_index_map = dict(sorted(index_map.items(), key=lambda item: item[1]))
    
    new_state = ''
    for old_index in sorted_index_map.keys():
        new_state += state[old_index]

    return int(new_state, base=2)

def calculate_swap_matrix(index_map: dict, circuit_qubits: list[Qubit]) -> np.ndarray:
    """Calculates the swap matrix for rearranging qubits based on an index map.

    This function calculates the swap matrix that rearranges qubits in a circuit
    based on a given index map. The index map specifies how qubits should be shifted
    to new positions.

    Parameters
    ----------
    index_map : dict
        A dictionary that maps old qubit indices to new qubit indices.
    circuit_qubits : list of Qubit
        The list of qubits in the circuit.

    Returns
    -------
    np.ndarray
        The swap matrix for rearranging qubits.
    """
    swap_matrix = np.identity(2**len(circuit_qubits))

    swapped_cols = set()  # Keep track of columns that have been swapped

    for col in range(2**len(circuit_qubits)):
        if col in swapped_cols:
            continue  # Skip columns that have already been swapped

        new_col = calculate_new_col(col, index_map)
        # print(f'Mapping column {col} to {new_col}')
        
        # Swap columns
        tmp = swap_matrix[col].copy()  # Create a copy of the column
        swap_matrix[col] = swap_matrix[new_col]
        swap_matrix[new_col] = tmp

        # Update swapped_cols
        swapped_cols.add(col)
        swapped_cols.add(new_col)

    return swap_matrix

def calculate_gate_matrix(gate: Gate, qubits: list[Qubit], circuit_qubits: list[Qubit]):
    """Calculates the global gate matrix for the given gate and qubits in a circuit.

    This function calculates the global gate matrix for a specified gate and its associated qubits
    in a circuit. The calculation involves considering the qubit positions, handling swaps if necessary,
    and constructing the final global gate matrix.

    Parameters
    ----------
    gate : Gate
        The quantum gate to be applied.
    qubits : list of Qubit
        The qubits on which the gate is applied.
    circuit_qubits : list of Qubit
        All qubits in the circuit.

    Returns
    -------
    np.ndarray
        The resulting global gate matrix.
    """
    # determine the global matrix for this gate:
    global_gate_matrix = np.identity(2**len(circuit_qubits), dtype=complex)
    
    index_map = {i: i for i in range(len(circuit_qubits))}

    if len(qubits) == 2:
        # determine the indices
        qb0_index = circuit_qubits.index(qubits[0])
        qb1_index = circuit_qubits.index(qubits[1])

        # if the two qubits are not adjacend, transfer the quantum information
        # using swap gates
        if np.abs(qb0_index - qb1_index) != 1:
            new_qb1_index = qb0_index + 1
            index_map = {}
            for i in range(len(circuit_qubits)):
                if i <= qb0_index:
                    # qubits prior to qb0 stay in place
                    index_map[i] = i
                elif i == qb1_index:
                    # qb1 is mapped to its new position
                    index_map[qb1_index] = new_qb1_index
                else:
                    # qubits after qb0 (except qb1) are shifted one to the right
                    index_map[i] = i + 1
    
    # replace the submatrices for a certain qubit
    locals = [] # list of local matrices
    i = 0
    used_qbs = set() # keep track of qubits that participated in a gate
    while i < len(circuit_qubits):
        qb = circuit_qubits[i]
        i += 1
        if qb in used_qbs:
            continue
        if qb in qubits:
            locals.append(gate.to_matrix())

            # add all qubits of the gate to used_qubits
            for q in qubits:
                used_qbs.add(q)
        else:
            locals.append(np.identity(2, dtype=complex))

    if len(locals) > 1:
        global_gate_matrix = np.kron(locals[0], locals[1])
        for local in locals[2:]:
            global_gate_matrix = np.kron(global_gate_matrix, local)
    else:
        global_gate_matrix = locals[0]
    
    swap_matrix = calculate_swap_matrix(index_map, circuit_qubits)

    return np.matmul(swap_matrix, np.matmul(global_gate_matrix, swap_matrix))

def calculate_circuit_matrix(circuit: QuantumCircuit):
    """Calculates the matrix representation of a quantum circuit.

    This function calculates the matrix representation of a given quantum circuit.
    It applies the gates in the circuit's data list to construct the circuit matrix.

    Parameters
    ----------
    circuit : QuantumCircuit
        The quantum circuit to calculate the matrix for.

    Returns
    -------
    np.ndarray
        The matrix representation of the quantum circuit.
    """
    circuit_matrix = np.identity(2**len(circuit.qubits))
    for instruction in circuit.data[::-1]:
        gate: Gate = instruction[0]
        qubits: list[Qubit] = instruction[1:][0]

        global_gate_matrix = calculate_gate_matrix(gate, qubits, circuit.qubits)
        # print_matrix(global_gate_matrix)
        circuit_matrix = np.matmul(global_gate_matrix, circuit_matrix)
    
    return circuit_matrix

circuit_matrix = calculate_circuit_matrix(circ)
print('Circuit matrix: ')
print_matrix(np.round(circuit_matrix, 10))

input_state = np.zeros(2**num_qb, dtype=complex)
input_state[0] = 1

output_state = np.array([1, 0])# np.matmul(circuit_matrix, input_state)
print('expected output state: ')
print_matrix(np.round(output_state, 10))
# fidelity
f = np.matmul(np.conjugate(output_state), np.matmul(rho, output_state))
print(f'Calculated fidelity: {np.real(f):.5f}')

Circuit matrix: 


<IPython.core.display.Latex object>

expected output state: 


<IPython.core.display.Latex object>

Calculated fidelity: 1.00000
